<a href="https://colab.research.google.com/github/myunsong/AutoPortraitMatting/blob/master/smolagents_hf_daily_paper_summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install smolagents
!pip install huggingface_hub
!pip install requests beautifulsoup4 arxiv pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 39.9 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 7.7 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-a

In [ ]:
import os
from smolagents import tool, CodeAgent, HfApiModel
import requests
from bs4 import BeautifulSoup
import json
import arxiv
from pypdf import PdfReader
from google.colab import userdata
import time

# Access the token from secrets using userdata
hf_api_token = userdata.get('HF_API_TOKEN')

@tool
def get_hugging_face_top_daily_paper() -> str:
    """
    Retrieves the title of the most upvoted paper from Hugging Face's daily papers page.
    Returns:
        str: The title of the most upvoted paper.
    """
    try:
        url = "https://huggingface.co/papers"
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")
        containers = soup.find_all('div', class_='SVELTE_HYDRATER contents')
        top_paper = ""
        for container in containers:
            data_props = container.get('data-props', '')
            if data_props:
                try:
                    json_data = json.loads(data_props.replace('"', '"'))
                    if 'dailyPapers' in json_data:
                        top_paper = json_data['dailyPapers'][0]['title']
                except json.JSONDecodeError:
                    continue
        return top_paper
    except requests.exceptions.RequestException as e:
        print(f"Error occurred while fetching the HTML: {e}")
        return None

@tool
def get_paper_id_by_title(title: str) -> str:
    """
    Retrieves the arXiv paper ID by its title.
    Args:
        title: The paper title for which to get the ID.
    Returns:
        str: The arXiv paper ID, or a message if not found.
    """
    try:
        search = arxiv.Search(query=title, max_results=1)
        results = list(search.results())
        if not results:
            return f"No arXiv ID found for the paper titled '{title}'."
        return results[0].entry_id
    except Exception as e:
        return f"Error searching arXiv for '{title}': {e}"



@tool
def download_paper_by_id(paper_id: str) -> None:
    """
    Downloads a paper from arXiv using its ID and saves it locally as "paper_{paper_id}.pdf".
    Args:
        paper_id: The ID of the paper to download.
    """
    try:
        paper = next(arxiv.Client().results(arxiv.Search(id_list=[paper_id])))
        filename = f"paper_{paper_id}.pdf"
        paper.download_pdf(filename=filename)
        print(f"Downloaded paper to {filename}")
    except Exception as e:
        print(f"Error downloading paper {paper_id}: {e}")
        raise  # Re-raise the exception to halt the agent's progress


@tool
def read_pdf_first_paragraph(file_path: str) -> str:
    """
    Reads the first paragraph of a PDF file and returns its content as a string.
    Args:
        file_path: The path to the PDF file.
    Returns:
        str: The content of the first paragraph of the PDF.
    """
    try:
        reader = PdfReader(file_path)
        first_page_text = reader.pages[0].extract_text()

        # Split the text into paragraphs (assuming paragraphs are separated by two newlines)
        paragraphs = first_page_text.split('\n\n')

        # Return the first non-empty paragraph
        for paragraph in paragraphs:
            if paragraph.strip():
                return paragraph.strip()
        return "No valid paragraph found."
    except Exception as e:
        return f"Error reading PDF: {e}"

# Initialize the agent with tools
agent = CodeAgent(
    model=HfApiModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct", token=hf_api_token, timeout=120),
    tools=[get_hugging_face_top_daily_paper, get_paper_id_by_title, download_paper_by_id, read_pdf_first_paragraph],
    planning_interval=2,
    max_steps=10,
    additional_authorized_imports=["requests", "bs4", "json", "arxiv", "pypdf", "time"],
)

# Run the agent with the task
result = agent.run("Find the most upvoted paper on Hugging Face, download it, and extract the first paragraph. DO NOT EXECUTE THOUGHTS, ONLY FUNCTIONS !!")

# Inspect the state of the agent
# print("Agent State:", agent.state)

# Print the final result
print(f"Final Result:\n{result}")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find the most upvoted paper on Hugging Face, download it, and extract the first paragraph. DO NOT EXECUTE       │
│ THOUGHTS, ONLY FUNCTIONS !!                                                                                     │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here is the plan of action that I will follow to solve the task:
                ```
                1. Retrieve the title of the most upvoted paper from Hugging Face's daily papers page.
2. Use the title of the most upvoted paper to find its arXiv paper ID.
3. Download the paper using its arXiv paper ID.
4. Read the first paragraph from the downloaded PDF file.
5. Provide the first paragraph as the final answer.

<end_plan>
                ```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  most_upvoted_paper_title = get_hugging_face_top_daily_paper()                                                    
  print(most_upvoted_paper_title)                                                                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
SuperGPQA: Scaling LLM Evaluation across 285 Graduate Disciplines

Out: None

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 9 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(first_paragraph)                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: M-A-P
SuperGPQA: Scaling LLM Evaluation across
285 Graduate Disciplines
M-A-P
ByteDance.Inc, 2077.AI
https://supergpqa.github.io/
Abstract
Large language models (LLMs) have demonstrated remarkable proficiency in main-
stream academic disciplines such as mathematics, physics, and computer science. How-
ever, human knowledge encompasses over 200 specialized disciplines, far exceeding
the scope of existing benchmarks. The capabilities of LLMs in many of these spe-
cialized fields—particularly in light industry, agriculture, and service-oriented disci-
plines—remain inadequately evaluated. To address this gap, we present SuperGPQA,
a comprehensive benchmark that evaluates graduate-level knowledge and reasoning
capabilities across 285 disciplines. Our benchmark employs a novel Human-LLM col-
laborative filtering mechanism to eliminate trivial or ambiguous questions through
iterative refinement based on both LLM responses and expert feedback. Our experi-
mental results reveal significant room for improvement in the performance of current
state-of-the-art LLMs across diverse knowledge domains ( e.g., the reasoning-focused
model DeepSeek-R1 achieved the highest accuracy of 61.82% on SuperGPQA), high-
lighting the considerable gap between current model capabilities and artificial general
intelligence. Additionally, we present comprehensive insights from our management
of a large-scale annotation process, involving over 80 expert annotators and an interac-
tive Human-LLM collaborative system, offering valuable methodological guidance for
future research initiatives of comparable scope.
Figure 1. Benchmark Comparison. Left: Radar Chart. Discrimination: The degree of
distinction between different models (detailed in Sec. 4.4). Climbing Space: The remaining
improvement space for the SOTA models. Corr. with Arena: Correlation with Chatbot Arena
Elo scores. Right: Performance comparison of SOTA models across different benchmarks.
arXiv:2502.14739v1  [cs.CL]  20 Feb 2025

[Step 8: Duration 41.63 seconds| Input tokens: 42,071 | Output tokens: 968]

Final Result:
M-A-P
SuperGPQA: Scaling LLM Evaluation across
285 Graduate Disciplines
M-A-P
ByteDance.Inc, 2077.AI
https://supergpqa.github.io/
Abstract
Large language models (LLMs) have demonstrated remarkable proficiency in main-
stream academic disciplines such as mathematics, physics, and computer science. How-
ever, human knowledge encompasses over 200 specialized disciplines, far exceeding
the scope of existing benchmarks. The capabilities of LLMs in many of these spe-
cialized fields—particularly in light industry, agriculture, and service-oriented disci-
plines—remain inadequately evaluated. To address this gap, we present SuperGPQA,
a comprehensive benchmark that evaluates graduate-level knowledge and reasoning
capabilities across 285 disciplines. Our benchmark employs a novel Human-LLM col-
laborative filtering mechanism to eliminate trivial or ambiguous questions through
iterative refinement based on both LLM responses and expert feedback. Our experi-
mental results reveal s